<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/main/ex2_shablul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade plotly
pip install --upgrade plotly notebook jupyterlab

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [23]:
# 📚 Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from difflib import get_close_matches
import matplotlib.pyplot as plt
from collections import defaultdict
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display
import plotly.io as pio


from google.colab import output
output.enable_custom_widget_manager()
# ================================
# 🌐 Build the Index from mqtt.org
# ================================
links = {
    1: "https://mqtt.org/",
    2: "https://mqtt.org/getting-started/",
    3: "https://mqtt.org/software/",
}

custom_stop_words = {
        'a', 'an', 'the', 'and', 'or', 'but', 'if', 'then', 'else', 'for', 'on', 'in', 'with',
        'is', 'are', 'was', 'were', 'be', 'being', 'been',
        'of', 'to', 'from', 'by', 'as', 'at', 'it', 'its', 'that', 'this', 'these', 'those',
        'we', 'you', 'he', 'she', 'they', 'i', 'me', 'my', 'your', 'our', 'their',
        'which', 'who', 'whom', 'what', 'when', 'where', 'why', 'how', 'can', 'will', 'should', 'could','mqttorg'
        ,'png','faq'
    }

custom_stop_words.update([
    'mqtt', 'org', 'https', 'http', 'www', 'com', 'home',
    'page', 'site', 'learn', 'get', 'started', 'software'
])

def apply_stemming(self, word_counts):
        stemmed_index = defaultdict(int)
        for word, count in word_counts.items():
            stemmed_word = self.stemmer.stem(word)
            stemmed_index[stemmed_word] += count
        return dict(stemmed_index)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text
stemmer = PorterStemmer()
index = {}
doc_id_to_url = {}
# this is the crwaling codevisited = set()
visited = set()
to_visit = list(links.values())
index = {}
MAX_PAGES = 5

def is_valid_link(href):
    return href and href.startswith("https://mqtt.org") and '#' not in href

while to_visit and len(visited) < MAX_PAGES:
    url = to_visit.pop()
    if url in visited:
        continue
    visited.add(url)

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Failed to load {url}: {e}")
        continue

    doc_id = len(visited)
    doc_id_to_url[doc_id] = url
    text = clean_text(soup.get_text())
    words = text.split()

    for word in words:
        if word in custom_stop_words or len(word) < 3:
            continue
        stem = stemmer.stem(word)
        if stem not in index:
            index[stem] = set()
        index[stem].add(doc_id)

    # Crawl internal links
    for link_tag in soup.find_all('a', href=True):
        href = link_tag['href']
        full_url = href if href.startswith("http") else f"https://mqtt.org{href}"
        if is_valid_link(full_url) and full_url not in visited and full_url not in to_visit:
            to_visit.append(full_url)


# up to here


data = {"term": [], "DocURLs": []}
for term, doc_ids in index.items():
    urls = [doc_id_to_url[doc_id] for doc_id in sorted(doc_ids)]
    data["term"].append(term)
    data["DocURLs"].append(urls)

df_index = pd.DataFrame(data)

# ================================
# 🎨 UI Styles (White Background, Black Text)
# ================================
def apply_styles():
    display(HTML("""
    <style>
    body, .output_area {
        background-color: white !important;
        color: black !important;
    }
    .widget-label { font-weight: bold; font-size: 16px; color: black; }
    .widget-button {
        background-color: #4CAF50;
        color: white;
        font-size: 16px;
        border-radius: 8px;
        padding: 8px 16px;
    }
    .widget-text input {
        padding: 8px;
        font-size: 16px;
        border-radius: 8px;
        border: 1px solid #ccc;
        color: black;
        background-color: white;
    }
    </style>
    """))

apply_styles()

# ================================
# 📋 Screens Definitions
# ================================

def show_admin_dashboard():
    clear_output()
    display(main_menu)
    display(HTML("<h2 style='color:black;'>🛠️ Admin Dashboard</h2>"))
    add_term = widgets.Text(description="Add Term:", layout=widgets.Layout(width='400px'))
    add_doc = widgets.Text(description="Add DocID:", layout=widgets.Layout(width='400px'))
    add_button = widgets.Button(description="Add Entry", button_style='success')

    def on_add_click(b):
        term = add_term.value.strip()
        doc = add_doc.value.strip()
        if term and doc:
            if term in df_index['term'].values:
                df_index.loc[df_index['term'] == term, 'DocURLs'].values[0].append(doc)
            else:
                df_index.loc[len(df_index.index)] = [term, [doc]]
            display(HTML("<b style='color:green;'>Entry Added Successfully!</b>"))
            display(df_index)
        else:
            display(HTML("<b style='color:red;'>Please provide both Term and DocID.</b>"))

    add_button.on_click(on_add_click)
    display(add_term, add_doc, add_button)


def show_search_screen():
    clear_output()
    display(main_menu)
    display(HTML("<h2 style='color:black;'>🔍 Search Engine</h2>"))

    search_term = widgets.Text(description="Search Term:", layout=widgets.Layout(width='400px'))
    search_button = widgets.Button(description="Search", button_style='primary')
    smart_search_button = widgets.Button(description="Smart Search", button_style='info')
    results_box = widgets.VBox()

    def display_results(result_df):
        """Display results in styled cards"""
        result_widgets = []
        for _, row in result_df.iterrows():
            card = widgets.HTML(
                value=f"""
                <div style="
                    background-color: #ffffff;
                    padding: 15px;
                    border-radius: 10px;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                    margin-bottom: 10px;
                    color: black;
                    ">
                    <h3 style="color:#4CAF50;">📚 Term: {row['term']}</h3>
                    <p><b>Documents:</b><br>
                    {''.join(f"<a href='{url}' target='_blank'>{url}</a><br>" for url in row['DocURLs'])}
                    </p>
                </div>
                """
            )
            result_widgets.append(card)
        if result_widgets:
            results_box.children = result_widgets
        else:
            results_box.children = [widgets.HTML("<b style='color:red;'>No results found.</b>")]

    def on_search_click(b):
        term = search_term.value.strip()
        if term:
            result_df = df_index[df_index['term'] == term]
            display_results(result_df)
        else:
            results_box.children = [widgets.HTML("<b style='color:red;'>Please enter a term to search.</b>")]

    def on_smart_search_click(b):
        term = search_term.value.strip()
        matches = get_close_matches(term, df_index['term'].tolist(), n=1, cutoff=0.6)
        if matches:
            corrected = matches[0]
            corrected_df = df_index[df_index['term'] == corrected]
            display(HTML(f"<b>Did you mean '<span style='color:orange;'>{corrected}</span>'?</b>"))
            display_results(corrected_df)
        else:
            results_box.children = [widgets.HTML("<b style='color:red;'>No similar terms found.</b>")]

    search_button.on_click(on_search_click)
    smart_search_button.on_click(on_smart_search_click)

    display(search_term, search_button, smart_search_button, results_box)


def show_statistics_screen():
    clear_output()
    display(main_menu)
    display(HTML("<h2 style='color:black;'>📊 System Statistics Dashboard</h2>"))

    total_terms = len(df_index)
    avg_docs = df_index['DocURLs'].apply(len).mean()
    max_docs = df_index['DocURLs'].apply(len).max()
    unique_docs = set([doc for sublist in df_index['DocURLs'] for doc in sublist])
    total_docs = len(unique_docs)

    stats_html = f"""
    <div style="display: flex; justify-content: space-around; flex-wrap: wrap; font-size:16px; text-align:center; margin-top:20px; color: black;">
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">📚</div>
            <div style="font-size: 20px;"><b>{total_terms}</b></div>
            <div>Unique Terms</div>
        </div>
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">📄</div>
            <div style="font-size: 20px;"><b>{total_docs}</b></div>
            <div>Unique Documents</div>
        </div>
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">📈</div>
            <div style="font-size: 20px;"><b>{round(avg_docs, 2)}</b></div>
            <div>Avg Docs per Term</div>
        </div>
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">🔝</div>
            <div style="font-size: 20px;"><b>{max_docs}</b></div>
            <div>Max Docs per Term</div>
        </div>
    </div>
    """
    display(HTML(stats_html))
    dropdown = widgets.Dropdown(
        options=["Sunburst","Pie Chart", "Bar Chart"],
        value="Sunburst",
        description="Chart:",
        style={'description_width': '150px'},
        layout=widgets.Layout(width='300px')
    )
    display(dropdown)
    out = widgets.Output()
    display(out)
    def redraw(change):
        with out:
            out.clear_output()
            if change["new"] == "Pie Chart":
                plt.figure(figsize=(6,6))
                doc_counts = df_index['DocURLs'].apply(len).value_counts().sort_index()
                plt.pie(doc_counts, labels=doc_counts.index, autopct='%1.1f%%', startangle=140)
                plt.title('Documents per Term Distribution')
                plt.show()
            if change["new"] == "Bar Chart":
                top_terms = df_index.assign(doc_count=df_index['DocURLs'].apply(len)) \
                                     .sort_values('doc_count', ascending=False) \
                                     .head(50)
                plt.figure(figsize=(8,5))
                plt.bar(top_terms['term'], top_terms['doc_count'])
                plt.xlabel('Term')
                plt.ylabel('Number of Documents')
                plt.title('Top 50 Terms by Document Count')
                plt.xticks(rotation=45)
                plt.show()
            if change["new"] == "Sunburst":
                # Prepare data for sunburst
                labels = []
                parents = []
                values = []

                # Initialize groupings BEFORE root node value is calculated
                grouped = {}
                group_values = {}

                # Group terms by first letter
                for term, docs in index.items():
                    if len(term) <= 2:
                        continue
                    if len(docs) <= 1:
                        continue
                    if not term.isalpha():
                        continue

                    first_letter = term[0].upper()
                    grouped.setdefault(first_letter, []).append((term, len(docs)))
                    group_values[first_letter] = group_values.get(first_letter, 0) + len(docs)

                # Root node
                labels.append("Words")
                parents.append("")
                values.append(sum(group_values.values()) or 1)  # fallback to 1 if empty

                # Group nodes and terms
                for letter, terms in grouped.items():
                    labels.append(letter)
                    parents.append("Words")
                    values.append(group_values[letter])

                    for term, freq in terms:
                        labels.append(term)
                        parents.append(letter)
                        values.append(freq)

                fig = go.Figure()
                fig.add_trace(go.Sunburst(
                    labels=labels,
                    parents=parents,
                    values=values,
                    branchvalues="total",
                    maxdepth=2
                ))
                fig.update_layout(
                    grid=dict(columns=2, rows=1),
                    margin=dict(t=0, l=0, r=0, b=0),
                    height=500
                )

                pio.show(fig)

    redraw({"new": dropdown.value})
    dropdown.observe(redraw, names="value")


# ================================
# 🚀 Main Menu with Navigation
# ================================
menu_title = widgets.HTML("<h1 style='text-align:center; color:black;'>📈 Business Statistics Dashboard</h1>")

admin_button = widgets.Button(description="Admin Dashboard",
                              layout=widgets.Layout(width='250px'),
                              button_style='primary')

search_button = widgets.Button(description="Search Engine",
                               layout=widgets.Layout(width='250px'),
                               button_style='primary')

stats_button = widgets.Button(description="Statistics",
                              layout=widgets.Layout(width='250px'),
                              button_style='primary')

admin_button.on_click(lambda b: show_admin_dashboard())
search_button.on_click(lambda b: show_search_screen())
stats_button.on_click(lambda b: show_statistics_screen())

main_menu = widgets.VBox([menu_title, admin_button, search_button, stats_button],
                         layout=widgets.Layout(align_items='center'))


show_admin_dashboard()


Dropdown(description='Chart:', layout=Layout(width='300px'), options=('Sunburst', 'Pie Chart', 'Bar Chart'), s…

Output()